In [1]:
from azureml.core import Workspace

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
#load the dataset to a pandas dataframe
credit_card_data = pd.read_csv("creditcard.csv")

In [4]:
#first 5 rows of the datasets
credit_card_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
#Info about the dataset
credit_card_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

In [6]:
#Checking the no. of missing values in each columns
credit_card_data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
#Checking the distiribution of legit and fraudulent transactions
credit_card_data["Class"].value_counts()

0    284315
1       492
Name: Class, dtype: int64

**This dataset is highly unbalanced

0 => Normal Transactions

1=> Fraudulent Transcations**

In [8]:
#Seperating the data for analysis
legit = credit_card_data[credit_card_data.Class==0]
fraud = credit_card_data[credit_card_data.Class==1]

In [9]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [10]:
#Statistical measures of the data
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [11]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [12]:
#Compare the vlaues for both tansactions
credit_card_data.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [13]:
legit_sample = legit.sample(n=492)
legit_sample

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
53610,46072.0,1.156220,0.194999,0.364930,0.583928,-0.308345,-0.416604,-0.153435,0.121860,-0.103941,...,-0.205932,-0.641594,0.183113,0.158497,0.054791,0.097237,-0.016128,0.017208,1.29,0
91713,63595.0,1.245674,0.166975,0.488306,0.635322,-0.562777,-1.011073,0.014953,-0.160211,0.170362,...,-0.262581,-0.816264,0.140304,0.357827,0.186423,0.096544,-0.035866,0.018495,8.99,0
9424,13869.0,1.034913,-0.220800,1.445937,1.067299,-0.855386,0.489859,-0.859498,0.259974,2.227795,...,-0.155710,0.002047,0.129711,0.063100,0.022136,0.316166,0.018146,0.020191,28.93,0
74211,55471.0,-0.598879,0.985480,-0.126184,-1.240506,2.401570,3.527533,-0.158224,0.155543,-0.453572,...,0.480494,-1.164211,0.052249,0.968973,-0.017450,0.092729,0.302860,0.121837,0.99,0
240746,150731.0,1.275429,-1.701347,-3.229386,-0.561080,2.022496,3.304602,0.087475,0.608151,0.211957,...,0.457165,0.385004,-0.275776,0.798240,-0.058068,0.763898,-0.148800,-0.005147,417.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91970,63711.0,-0.115339,-0.544470,1.900428,-1.915342,-1.281465,0.281873,-0.859227,0.071587,-1.574438,...,-0.204746,-0.053434,-0.160547,-0.442585,-0.057388,-0.197823,-0.022131,0.031871,24.56,0
76341,56521.0,0.965416,-0.511037,1.215316,0.745256,-0.617781,1.500911,-1.084462,0.763349,1.189166,...,0.028143,0.322667,0.220823,-0.641547,-0.171418,0.431120,0.075350,0.012096,11.50,0
25781,33763.0,-0.299301,0.009072,1.010398,-2.020044,0.528802,-0.442317,0.601823,-0.193932,-1.635754,...,0.310641,0.876911,-0.389318,-0.271350,0.511619,-0.192328,-0.196310,-0.247240,12.95,0
226777,144799.0,2.042562,-0.083512,-1.366869,0.169494,0.167931,-0.737410,0.082419,-0.195797,0.395743,...,0.295288,0.996439,-0.046098,-0.356959,0.253081,-0.095809,-0.017076,-0.071631,1.00,0


In [14]:
new_dataset = pd.concat([legit_sample, fraud], axis = 0)

In [15]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
53610,46072.0,1.156220,0.194999,0.364930,0.583928,-0.308345,-0.416604,-0.153435,0.121860,-0.103941,...,-0.205932,-0.641594,0.183113,0.158497,0.054791,0.097237,-0.016128,0.017208,1.29,0
91713,63595.0,1.245674,0.166975,0.488306,0.635322,-0.562777,-1.011073,0.014953,-0.160211,0.170362,...,-0.262581,-0.816264,0.140304,0.357827,0.186423,0.096544,-0.035866,0.018495,8.99,0
9424,13869.0,1.034913,-0.220800,1.445937,1.067299,-0.855386,0.489859,-0.859498,0.259974,2.227795,...,-0.155710,0.002047,0.129711,0.063100,0.022136,0.316166,0.018146,0.020191,28.93,0
74211,55471.0,-0.598879,0.985480,-0.126184,-1.240506,2.401570,3.527533,-0.158224,0.155543,-0.453572,...,0.480494,-1.164211,0.052249,0.968973,-0.017450,0.092729,0.302860,0.121837,0.99,0
240746,150731.0,1.275429,-1.701347,-3.229386,-0.561080,2.022496,3.304602,0.087475,0.608151,0.211957,...,0.457165,0.385004,-0.275776,0.798240,-0.058068,0.763898,-0.148800,-0.005147,417.00,0


In [16]:
new_dataset.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
279863,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
280143,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
280149,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [17]:
new_dataset["Class"].value_counts()

1    492
0    492
Name: Class, dtype: int64

In [18]:
new_dataset.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,96779.691057,0.017658,0.107751,-0.029253,0.001924,-0.077603,-0.023648,-0.024268,-0.021448,-0.021615,...,-0.001016,-0.030228,-0.003444,0.005157,0.024496,0.010761,0.053355,0.003298,0.009018,80.034004
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [19]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config()

**Splitting the data into features and targets**

In [20]:
X = new_dataset.drop(columns = 'Class', axis = 1)
Y = new_dataset['Class']
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
53610    46072.0  1.156220  0.194999  0.364930  0.583928 -0.308345 -0.416604   
91713    63595.0  1.245674  0.166975  0.488306  0.635322 -0.562777 -1.011073   
9424     13869.0  1.034913 -0.220800  1.445937  1.067299 -0.855386  0.489859   
74211    55471.0 -0.598879  0.985480 -0.126184 -1.240506  2.401570  3.527533   
240746  150731.0  1.275429 -1.701347 -3.229386 -0.561080  2.022496  3.304602   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [21]:
print(Y)

53610     0
91713     0
9424      0
74211     0
240746    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


**Splitting the data into training and test data**

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state =2)

In [23]:
Y_train

100623    1
176026    0
254560    0
276688    0
189002    0
         ..
66861     0
166797    0
223618    1
43061     1
232841    0
Name: Class, Length: 787, dtype: int64

In [24]:
print(X.shape, X_train.shape, X_test.shape)

(984, 30) (787, 30) (197, 30)


In [25]:
print(Y.shape, Y_train.shape, Y_test.shape)

(984,) (787,) (197,)


**Configuring AutoML settings**

In [26]:
import logging

automl_settings = {
    "iteration_timeout_minutes": 10,
    "experiment_timeout_hours": 0.6,
    "enable_early_stopping": False,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
    "n_cross_validations": 5
}

In [27]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(task='classification',
                             X = X_train,
                             y = Y_train,
                             blocked_models=['SGD','MultinomialNaiveBayes','BernoulliNaiveBayes',
                                                'SVM','KNN','AveragedPerceptionClassifier',
                                                'DecissionTree','RandomForest','ExtremeRandomForest',
                                                'LightGBM','GradientBoosting','XGBoostClassifier',
                                                'LinearSVM'],
                             **automl_settings)

#Running the experiment

In [28]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, "Credit-Card")
local_run = experiment.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
Credit-Card,AutoML_631207ea-e024-420c-a5e8-efbb263a4dc1,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


In [29]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [35]:
#Getting the best model
best_run, fitted_model = local_run.get_output()
print(best_run)

Run(Experiment: Credit-Card,
Id: AutoML_631207ea-e024-420c-a5e8-efbb263a4dc1_76,
Type: None,
Status: Completed)


In [36]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
)), ('logisticregression', LogisticRegression(C=0.0062505519252739694, class_weight='balanced', dual=False, fit_intercept=True, intercept_scaling=1, l1_ratio=None, max_iter=100, multi_class='multinomial', n_jobs=1, penalty='l2', random_state=None, solver='saga', tol=0.0001, verbose=0, warm_start=False))], verbose=False))], flatten_transform=None, weights=[0.07142857142857142, 0.2857142857142857, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.07142857142857142, 0.14285714285714285, 0.07142857142857142, 0.07142857142857142]))],
         verbose=False)


In [31]:
y_predict = fitted_model.predict(X_test)
print(y_predict[:10])

[1 0 0 0 1 0 1 1 0 1]


In [32]:
from sklearn.metrics import mean_squared_error
from math import sqrt

y_actual = Y_test.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))
rmse

0.2759386380695814

In [33]:
sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)

Model MAPE:
0.15306122448979592

Model Accuracy:
0.846938775510204


**Registering and Deploying the model**

In [37]:
model_name = best_run.properties["model_name"]
registered_model = local_run.register_model(model_name=model_name, description = "AutoML Credit card fraud detection", tags=None)

In [38]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

In [39]:
best_run.download_file("outputs/scoring_file_v_1_0_0.py","inference/score.py")

In [41]:
from azureml.automl.core.shared import constants
best_run.download_file(constants.CONDA_ENV_FILE_PATH, "myenv.yml")
env = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

In [51]:
inference_config = InferenceConfig(entry_script= "inference/score.py", environment=env)
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description= "Credit Card Fraud Detection")
service = Model.deploy(ws, "automl-fraud-detection", [registered_model], inference_config, aciconfig)
service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-01-20 19:49:12+00:00 Creating Container Registry if not exists..
2022-01-20 19:59:12+00:00 Registering the environment..
2022-01-20 19:59:17+00:00 Building image..
2022-01-20 20:11:50+00:00 Generating deployment configuration.
2022-01-20 20:11:51+00:00 Submitting deployment to compute..
2022-01-20 20:12:02+00:00 Checking the status of deployment automl-fraud-detection..
2022-01-20 20:17:07+00:00 Checking the status of inference endpoint automl-fraud-detection.
Succeeded
ACI service creation operation finished, operation "Succeeded"
